# Lesson 3: Reflection and Blogpost Writing

## Setup

In [59]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [60]:
task = '''
        Write a concise but engaging blogpost about
       Rasheed Abdulkareem Data Expert Make sure the blogpost is
       within 1000 words.
       '''


## Create a writer agent

In [61]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [62]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [63]:
print(reply)

Title: Unveiling the Data Wizard: The Rise of Rasheed Abdulkareem

In the realm of data analytics, one name stands out among the rest - Rasheed Abdulkareem. With a keen eye for patterns and a passion for unraveling insights, Rasheed has carved a niche for himself as a data expert extraordinaire. His journey from humble beginnings to becoming a sought-after industry figure is nothing short of inspiring.

Rasheed's fascination with data started at a young age when he would spend hours poring over spreadsheets and datasets, trying to make sense of the numbers. What began as a hobby soon turned into a full-fledged passion, driving him to pursue a degree in data science and analytics.

After graduating, Rasheed wasted no time in making his mark in the industry. His unique approach to data analysis, coupled with his ability to simplify complex concepts, quickly caught the attention of both peers and clients. Before long, he was consulting for top firms, helping them harness the power of data

## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [64]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [65]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Rasheed Abdulkareem Data Expert Make sure the blogpost is
       within 1000 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Data Wizard: The Rise of Rasheed Abdulkareem

In the realm of data analytics, one name stands out among the rest - Rasheed Abdulkareem. With a keen eye for patterns and a passion for unraveling insights, Rasheed has carved a niche for himself as a data expert extraordinaire. His journey from humble beginnings to becoming a sought-after industry figure is nothing short of inspiring.

Rasheed's fascination with data started at a young age when he would spend hours poring over spreadsheets and datasets, trying to make sense of the numbers. What began as a hobby soon turned into a full-fledged passion, driving him to pursue a degree in data science and analytics.

After graduating, Rasheed wasted n


--------------------------------------------------------------------------------


## Nested chat

In [66]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [67]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [68]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [69]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [70]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [71]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [72]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Rasheed Abdulkareem Data Expert Make sure the blogpost is
       within 1000 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Data Wizard: The Rise of Rasheed Abdulkareem

In the realm of data analytics, one name stands out among the rest - Rasheed Abdulkareem. With a keen eye for patterns and a passion for unraveling insights, Rasheed has carved a niche for himself as a data expert extraordinaire. His journey from humble beginnings to becoming a sought-after industry figure is nothing short of inspiring.

Rasheed's fascination with data started at a young age when he would spend hours poring over spreadsheets and datasets, trying to make sense of the numbers. What began as a hobby soon turned into a full-fledged passion, driving him to pursue a degree in data science and analytics.

After graduating, Rasheed wasted n


--------------------------------------------------------------------------------
Legal Reviewer (to Critic):

As a Legal Reviewer:

- Ensure that any claims made about Rasheed Abdulkareem's expertise and achievements are factually accurate and can be substantiated.
- Review for any potential copyright infringement in the content, especially if any quotes or references have been used without proper attribution.
- Confirm that there are no defamatory statements or misleading information that could harm Rasheed Abdulkareem's reputation or legal standing.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

Review the following content. 
            

 Title: Unveiling the Data Wizard: The Rise of Rasheed Abdulkareem

In the realm of data analyti


--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Data Wizard: Rasheed Abdulkareem's Journey in Data Analytics

Meta Description: Explore the inspiring journey of Rasheed Abdulkareem, a prominent figure in the data analytics field, and uncover the secrets behind his success. Follow along as we delve into his expertise, passion for data, and contributions to the industry.

In the dynamic realm of data analytics, Rasheed Abdulkareem shines as a true wizard, captivating audiences with his exceptional skills and innovative insights. From humble beginnings to becoming a sought-after data expert, Rasheed's journey is both inspiring and impactful.

Rasheed's fascination with data began early, leading him to immerse himself in spreadsheets and datasets, laying the foundation for his stellar career in data science and analytics. His commitment to understanding intricate patterns and deriving meaningful conclusions set him

## Get the summary

In [73]:
print(res.summary)

Title: Unveiling the Data Wizard: Rasheed Abdulkareem's Journey in Data Analytics

Meta Description: Explore the inspiring journey of Rasheed Abdulkareem, a prominent figure in the data analytics field, and uncover the secrets behind his success. Follow along as we delve into his expertise, passion for data, and contributions to the industry.

In the dynamic realm of data analytics, Rasheed Abdulkareem shines as a true wizard, captivating audiences with his exceptional skills and innovative insights. From humble beginnings to becoming a sought-after data expert, Rasheed's journey is both inspiring and impactful.

Rasheed's fascination with data began early, leading him to immerse himself in spreadsheets and datasets, laying the foundation for his stellar career in data science and analytics. His commitment to understanding intricate patterns and deriving meaningful conclusions set him apart, garnering admiration from peers and clients alike.

Embracing his passion, Rasheed swiftly made